In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter#Counter计数器，统计字符出现的个数
from string import punctuation
import jieba
import jieba.analyse
import jieba.posseg as pseg
import codecs, sys

In [2]:
#with open('E:/维基百科中文20190720/extracted/AA/wiki_02_jian.txt','r',encoding='utf-8') as f:
#    data = pd.read_table(f)
#data.head(10)

数据预处理

In [3]:
# 英文标点符号+中文标点符号
punc = punctuation + u'.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s::a-zA-Z0-9:「」'
def clean(line):
    # we will learn the regular expression next course.
    line_re = re.sub(r"[{}]+".format(punc),"",line)
    return line_re

分词

In [33]:
STOP_WORDS = nltk.corpus.stopwords.words()
content = []
#target = codecs.open("E:/维基百科中文20190720/extracted/AA/wiki_02_jian_seg.txt", 'w', encoding="utf-8")   
with open('E:/维基百科中文20190720/extracted/AA/wiki_02_jian.txt', "r", encoding='utf-8-sig') as f:
    for line in f.readlines():#一行一行读
        if len(line)<=1 or line.startswith(">doc") or line.startswith("/doc"):continue
        line_clean = clean(line)
        line_seg = " ".join(jieba.cut(line_clean))
        line_seg_w = line_seg.split(" ")
        for word in line_seg_w:
            if word in STOP_WORDS :
                line_seg_w.remove(word) 
        content.append(line_seg_w)
        
        #target.writelines(line_seg)
f.close()
#target.close()

In [34]:
len(content)

1814671

词向量

In [35]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
path = get_tmpfile("word2vec.model")

model = Word2Vec(content, size=100, window=5, min_count=1, workers=4)
model.save("word2vec_02.model")

In [36]:
model = Word2Vec.load("word2vec_02.model")#加载

In [70]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [72]:
#tsne_plot(model)

In [133]:
model.train([["张伟","大家好"]], total_examples=model.corpus_count, epochs=model.iter)#model.train()？？只可以设置增加新语料，设置迭代次

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(10, 20)

In [31]:
model.wv["遵义市"]  # numpy vector of a word

array([-2.29001627e-03,  7.62209436e-03, -7.33958371e-03, -9.67524829e-04,
        9.89851542e-03, -3.07188420e-05,  5.14899474e-03,  8.15794151e-03,
        3.42693296e-03,  5.07293455e-03,  5.70513168e-03, -1.38736945e-02,
       -2.85584526e-03, -6.81061670e-03,  1.27837071e-02,  6.17387798e-03,
        1.60650406e-02, -9.68953781e-03, -1.27714658e-02,  3.16178106e-04,
        9.21780738e-05,  3.82249523e-03, -5.90246264e-03, -7.74748530e-03,
        9.11066029e-03, -6.16263878e-03,  5.44559816e-03, -7.12375448e-04,
        2.05145963e-03, -6.08816859e-04, -1.91276502e-02,  4.73872013e-03,
        3.85069684e-03,  1.17760326e-03,  5.39134489e-03,  6.00508228e-03,
       -6.23274967e-03,  3.44593031e-03, -1.18283564e-02,  4.05947765e-04,
        1.65105313e-02,  9.44453385e-03, -1.63313635e-02, -8.37097224e-03,
       -4.47184220e-03,  1.71865406e-03, -3.61083518e-03, -7.41649745e-03,
        1.42203551e-02, -1.50657655e-03, -1.43458117e-02,  5.60824992e-03,
       -2.26507219e-03,  

In [37]:
model.most_similar("使用")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('采用', 0.7824817895889282),
 ('选用', 0.7084349393844604),
 ('用于', 0.6987842321395874),
 ('可用', 0.6889451742172241),
 ('用', 0.6875796318054199),
 ('运用', 0.6861847639083862),
 ('导入', 0.6486833095550537),
 ('用到', 0.6468229293823242),
 ('所用', 0.6466189026832581),
 ('添加', 0.6439599990844727)]